In [1]:
import openai
import instructor
import json
import os
import malaya
from pydantic import BaseModel, Field
from bs4 import BeautifulSoup
from concurrent.futures import ThreadPoolExecutor, as_completed
from unidecode import unidecode
import re

minimum_len = 15

def simple_cleaning(string):
    return re.sub(r'[ ]+', ' ', unidecode(string).replace('\n', ' ').replace('--', ' ').replace('/', ' ')).strip()

/home/husein/3.9/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/husein/3.9/lib/python3.9/site-packages/malaya/tokenizer.py:214: FutureWarning: Possible nested set at position 3397
  self.tok = re.compile(r'({})'.format('|'.join(pipeline)))
/home/husein/3.9/lib/python3.9/site-packages/malaya/tokenizer.py:214: FutureWarning: Possible nested set at position 3927
  self.tok = re.compile(r'({})'.format('|'.join(pipeline)))


In [2]:
openai.api_type = 'azure'
openai.api_base = 'https://nous.openai.azure.com/'
openai.api_version = '2023-07-01-preview'
openai.api_key = ''

In [3]:
instructor.patch()

In [4]:
engine = 'nous-16k'

In [5]:
from typing import List

class QA(BaseModel):
    question: str
    answer: str
        
class QAS(BaseModel):
    qa: List[QA]

In [6]:
def predict(text):
    try:
        messages = [
            {"role": "user", "content": text},
        ]
        response = openai.ChatCompletion.create(
            engine=engine,
            response_model=QAS,
            messages=messages,
            temperature=0.7,
            max_tokens=1024,
            top_p=0.95,
        )
        return response.dict()
    except Exception as e:
        return None

In [7]:
!mkdir paragraph-wikipedia

mkdir: cannot create directory ‘paragraph-wikipedia’: File exists


In [8]:
def generate(filename, l):
    if os.path.exists(filename):
        return
    
    s = f"""
paragraph ```
{l['paragraph']}
```

berdasarkan paragraph, jana soalan melayu dan jawapan melayu
"""
    s = s.strip()
    r = predict(s)
    l['qa'] = r

    with open(filename, 'w') as fopen:
        json.dump(l, fopen)

In [9]:
data = []
with open('paragraph-wiki.jsonl') as fopen:
    for l in fopen:
        data.append(json.loads(l))
        
len(data)

49793

In [10]:
# !rm -rf paragraph-wikipedia
# !mkdir paragraph-wikipedia

In [12]:
from tqdm import tqdm

max_worker = 3
for i in tqdm(range(0, len(data), max_worker)):
    b = data[i: i + max_worker]
    filenames = [(os.path.join('paragraph-wikipedia', f'{i + k}.json'), l) for k, l in enumerate(b)]
    with ThreadPoolExecutor(max_workers=max_worker) as executor:
        futures = {executor.submit(generate, *f): f for f in filenames}

        for future in as_completed(futures):
            future.result()

  0%|                                                 | 0/16598 [00:00<?, ?it/s]/tmp/ipykernel_2961592/342045862.py:14: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.4/migration/
  return response.dict()
100%|██████████████████████████████████| 16598/16598 [51:45:45<00:00, 11.23s/it]
